In [1]:
import os
import sys

sys.path.append("../../../../../")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
import copy
import torch
from datetime import datetime
from src.utils.helper import Config, color_print
from src.utils.load import load_model, load_data, save_checkpoint
from src.models.evaluate import (
    evaluate_model,
    get_sparsity,
    get_similarity,
    get_perplexity,
)
from src.utils.sampling import SamplingDataset
from src.pruning.prune_head import head_importance_prunning
from src.pruning.prune import prune_concern_identification
from src.utils.helper import report_to_df, append_nth_row

In [3]:
name = "IMDB"
device = torch.device("cuda:0")
checkpoint = None
batch_size = 16
num_workers = 4
num_samples = 16
ratio = 0.5
seed = 44
include_layers = ["intermediate", "output"]
exclude_layers = [
    "attention",
]

In [4]:
script_start_time = datetime.now()
print(f"Script started at: {script_start_time.strftime('%Y-%m-%d %H:%M:%S')}")

Script started at: 2024-10-22 17:15:13


In [5]:
config = Config(name, device)
num_labels = config.config["num_labels"]
model = load_model(config)

Loading the model.

{

'architectures'

: 

'bert'

...

'tokenizer_name'

: 

'textattack/bert-base-uncased-imdb'

}

The model textattack/bert-base-uncased-imdb is loaded.

In [6]:
train_dataloader, valid_dataloader, test_dataloader = load_data(
    config,
    batch_size=batch_size,
    num_workers=num_workers,
    do_cache=True,
)

Loading cached dataset IMDB.

train.pkl is loaded from cache.

valid.pkl is loaded from cache.

test.pkl is loaded from cache.

The dataset IMDB is loaded

...

,
 

'path'

: 

'imdb'

}

In [7]:
# print("Evaluate the original model")
# result = evaluate_model(model, config, test_dataloader)

In [8]:
result_list = []

for concern in range(config.num_labels):
    config.init_seed()
    positive_samples = SamplingDataset(
        train_dataloader,
        config,
        concern,
        num_samples,
        True,
        4,
        resample=False,
    )
    negative_samples = SamplingDataset(
        train_dataloader,
        config,
        concern,
        num_samples,
        False,
        4,
        resample=False,
    )
    all_samples = SamplingDataset(
        train_dataloader,
        config,
        200,
        num_samples,
        False,
        4,
        resample=False,
    )

    module = copy.deepcopy(model)

    head_importance_prunning(module, config, all_samples, ratio)

    prune_concern_identification(
        module,
        config,
        positive_samples,
        negative_samples,
        include_layers=include_layers,
        exclude_layers=exclude_layers,
        sparsity_ratio=ratio,
        keep_dim=True,
        method="structed",
    )

    print(f"Evaluate the pruned model {concern}")
    result = evaluate_model(module, config, test_dataloader, verbose=True)
    result_list.append(result)
    get_sparsity(module)

    get_similarity(model, module, valid_dataloader, concern, num_samples, config)
    print("original model's perplexity")
    get_perplexity(model, valid_dataloader, config)
    print("pruned model's perplexity")
    get_perplexity(module, valid_dataloader, config)

Total heads to prune: 72

tensor([[0.5385, 0.4080, 0.4691, 0.5637, 0.5390, 0.4439, 0.5190, 0.4272, 0.4888,
         0.4545, 0.6183, 0.3817],
        [0.3811, 0.7038, 0.2962, 0.2981, 0.6218, 0.3951, 0.5404, 0.3963, 0.4056,
         0.3963, 0.4869, 0.5001],
        [0.7018, 0.3968, 0.3042, 0.2946, 0.2830, 0.3394, 0.3822, 0.2859, 0.2958,
         0.7170, 0.3353, 0.3565],
        [0.5079, 0.3977, 0.3184, 0.5424, 0.3104, 0.6144, 0.3219, 0.4152, 0.3848,
         0.6896, 0.4527, 0.5941],
        [0.4527, 0.4084, 0.4214, 0.5190, 0.4054, 0.6017, 0.4455, 0.4555, 0.3966,
         0.4975, 0.6034, 0.5755],
        [0.4534, 0.3214, 0.2436, 0.4015, 0.5182, 0.3032, 0.3531, 0.5374, 0.4019,
         0.7564, 0.5319, 0.3514],
        [0.4782, 0.3763, 0.4695, 0.4171, 0.4990, 0.6237, 0.5380, 0.4421, 0.5104,
         0.5605, 0.4333, 0.6132],
        [0.5826, 0.5346, 0.5279, 0.4711, 0.5681, 0.4406, 0.4656, 0.3921, 0.4873,
         0.4912, 0.6079, 0.4727],
        [0.4072, 0.5928, 0.4339, 0.4284, 0.5270, 0.4948, 0.4331, 0.4388, 0.4204,

{(4, 0), (3, 4), (3, 1), (3, 7), (4, 6), (5, 1), (8, 0), (3, 10), (8, 3), (11, 2), (10, 6), (8, 6), (2, 2), (1, 0), (10, 9), (10, 3), (2, 5), (1, 3), (2, 11), (0, 11), (2, 8), (1, 9), (11, 8), (7, 7), (4, 2), (5, 6), (4, 8), (3, 6), (5, 3), (8, 2), (10, 2), (9, 4), (11, 1), (11, 7), (10, 5), (2, 4), (1, 2), (2, 1), (2, 7), (10, 8), (1, 5), (6, 1), (2, 10), (1, 8), (6, 7), (6, 10), (3, 2), (4, 1), (5, 2), (4, 4), (8, 8), (3, 8), (5, 5), (5, 11), (11, 0), (9, 9), (5, 8), (0, 7), (11, 3), (11, 9), (10, 1), (10, 7), (8, 10), (2, 3), (8, 7), (1, 7), (0, 5), (2, 6), (10, 10), (7, 5), (6, 3), (0, 1)}

Evaluate the pruned model 0

Evaluating the model:   0%|                                                                                   …

...

Kernel CKA non-concern: 0.07067499991043291

original model's perplexity

1.0879530906677246

pruned model's perplexity

1.9980603456497192

In [9]:
df_list = [report_to_df(df) for df in result_list]
new_df = append_nth_row(df_list)
csv_name = f"{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
new_df.to_csv(f"results/{csv_name}.csv", index=False)
print(csv_name)
new_df

2024-10-22_17-31-16

,class,precision,recall,f1-score,support
0,0,0.0,0.0,0.0000,12500
1,1,0.5,1.0,0.6667,12500
